In [1]:
import openpyxl
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [2]:
url="https://topai.tools/browse"

In [3]:
driver = webdriver.Chrome()
driver.get(url)

In [4]:
def scroll_to_bottom():
    last_height=driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [5]:
scroll_to_bottom()

In [6]:
source = driver.page_source
driver.quit()

In [27]:
excel=openpyxl.Workbook()

In [28]:
sheet=excel.active
sheet.title='Top AI Tools'

In [29]:
sheet.append(['S.No.', 'Tool Name', 'Url', 'Price', 'Tags', 'Use-case'])

In [30]:
# source=requests.get(url).text
# source.raise_for_status()

In [31]:
soup=BeautifulSoup(source, 'lxml')

In [32]:
tools_section = soup.find_all('div', class_="tool-wrapper")

In [33]:
i=1
for tools in tools_section:
    try:
        divs=tools.find_all('div')
        tool_name=tools.h5.text.split('\n')[1]
        urls=divs[1].find('a', class_="mx-2 rounded p-1")
        tool_url=urls['href']
        hyperlink_formula = f'=HYPERLINK("{tool_url}", "{tool_url}")'
        price=divs[1].small.span.text
        tag_list=tools.find_all('p', class_="px-1")[1].find_all('span', class_="badge")
        tags=''
        for tag in tag_list:
            tags=tags+tag.text+', '
        tags=tags.rstrip(', ')
        use_cases=divs[1].p.text
        sheet.append([i, tool_name, hyperlink_formula, price, tags, use_cases])
        i+=1
    except Exception as e:
        print(e)

In [34]:
excel.save('Top AI Tools.xlsx')